In [85]:
import math
import numpy as np
import sqlite3
import pandas as pd
from ckiptagger import WS

#Q: title?

####load doc from csv####
df = pd.read_csv("./Tokenized.csv")
paragraphs=list()
df = df[df['paragraph'].notnull()]
for d in df['paragraph']:
        d=' '.join(d.split("/"))
        paragraphs.append(d)
print(paragraphs)


####ld doc(form database)####
# news = sqlite3.connect('./爬蟲/News.db')
# df = pd.read_sql("SELECT * FROM News", news,  index_col='id')
# df = df[df['paragraph']!='']


####ws####
# ws = WS('./data')
# paragraphs = [' '.join(ws([p])[0]) for p in df.paragraph]
# print(paragraphs)

['傳出 有 家 醫院 醫護員 確診 此 衛生 局長 縣府 防疫 說明 記者 證實 確實 接獲 家 醫院 出現 醫護 確診 針對 院 感染 個案 展開 全面 處理 說 醫院門診 暫停 住院 相關 部門 只出 進 員工 也 依 風險 分層 做 採 開始 包括 醫護 醫護 清潔工 等 採 感染 情況 做 全面 掌握 也 針對 門診 清潔 消毒 也 表示 急診 洗 腎空 間 特別 維持 民眾 服務 完整 性 採 也 清 消採 檢 確保 區域 乾淨 等 疫 調 處理 告 段 落 再 說明', '國內 本土 疫情 多 爆發 新增 本土 除 產 發局 員工 確診 國民黨 立委 警分局 也 傳出 刑警 確診 另 有 民眾 反映 到 有 確診 足跡 咖啡 廳 用餐 咖啡 廳老闆 告知 坐 確診者 沒收 到 細胞 簡訊 衛生局 也 咖啡 廳 索取 實聯 制資料 質疑 足跡匡列 作業 鬆綁 警分局 爆 刑警 確診 人 送 證物 到 地檢署 入庫 衛生局 召開 記者 表示 相關 個案 疫 調 立委 辦公室 繼 助理 確診 也 陰轉 陽 移置 治療 衛生局 表示 個案 縣市個案 疫 調 針對 確診 場域 北市府 有 協助 加強 清 消 呼籲 員工 自我 健康 監測 另 有 市民 指出 去 咖啡 廳 咖啡廳 公告 有確診者 足跡 事後 咖啡廳 老闆 告知 坐 確診者 沒收 到 細胞 簡訊 通知 咖啡廳 老闆 也 說 衛生局 索取 實聯 制資訊 質疑 足跡匡列 鬆綁 衛生局 回應 只 要 收到 通報 個案 啟動 疫 調 機制 依照 程序匡列 相關 接觸者 做 風險 評估 環境清 消等 民眾 匡列 標準 清楚 等 問題 近 當地 衛生 單位 詢問', '北山 出海 口 發生 一起 民眾 落海 意外 楊姓 男子 疑 搭乘 船 出海 想到 不慎 落海 海岸 巡 消防隊 接獲 通報 趕 現場 救援 楊姓 男子 獲救 生命 跡象 送醫 治 後續 地檢署 檢察官 進行 相 驗 釐清 相關 案情 消防局 長 陳叁奇 表示 意外 發生 北山 出海 口 海 約 處 岸 巡 發現 船 原地 轉 船 無人 通報艇 前往 搜救 尋獲 落海 男子 救 楊姓 男子 呼吸 心跳 上 送 醫院 搶救 海巡艇 靠岸 身 防護服 人員 落海 男子 抬上 岸 消防 隊員 實施 CPR 心肺 復甦 術 恢復 生命跡象 行人 趕緊 人

In [113]:
#TF-IWF
import math
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

def TF_skl(paragraph):
    cv = CountVectorizer()
    cv_fit = cv.fit_transform(paragraph)
    word = cv.get_feature_names()
    count = np.asarray(cv_fit.sum(axis=0))[0]
    f = count/np.sum(count)
    tf = dict(zip(word, f))
    #print(tf)
    return tf

def IWF_skl(paragraphs):
    iwf = dict()
    median_iwf = 0
    total=0    #total words in corpus
    cv = CountVectorizer()
    cv_fit = cv.fit_transform(words)
    word = cv.get_feature_names()
    count = np.asarray(cv_fit.sum(axis=0))[0]
    iwf = dict(zip(word,  count))
    total += (count.sum(axis=0))
    values = []
    for k, v in iwf.items():
        iwf[k] = math.log(total / v, 10)
        values.append(math.log(total / v, 10))
    median_iwf = np.median(values)
    return iwf, median_iwf

# def IWF(paragraphs):
#     iwf = dict()
#     median_iwf = 0
#     for paragraph in paragraphs:
#         words = paragraph.split(" ")
#         for word in words:
#             if word not in iwf:
#                 iwf[word] = 1
#             else:
#                 iwf[word] += 1
#     total_words = len(iwf) #所有文章
#     values = []
#     for k, v in iwf.items():
#         iwf[k] = math.log(total_words / v, 10)
#         values.append(math.log(total_words / v, 10))
#     median_iwf = np.median(values)
#     return iwf, median_iwf

res = dict()
#iwf, median_iwf = IWF(paragraphs)
iwf, median_iwf = IWF_skl(paragraphs)
for p in paragraphs:
    words = p.split(" ")
    tf = TF_skl(words)
    #print(words)
    for word in words:
        if word in tf:
            if word not in iwf.keys():
                res[word] = tf[word] * median_iwf
            else:
                res[word] = tf[word] * iwf[word]
dict(sorted(res.items(), key=lambda item: item[1], reverse=True))
#dict(sorted(iwf.items(), key=lambda item: item[1], reverse=True))

{'士檢': 0.20411667451031362,
 '救護': 0.17735980706514143,
 '縣市': 0.14508293041174253,
 '鍋爐': 0.13846511253331215,
 '確診': 0.13607778300687576,
 '聯合': 0.13154185690664655,
 '新聞網': 0.13154185690664655,
 '讀者': 0.13154185690664655,
 '快速': 0.13154185690664655,
 '整理': 0.13154185690664655,
 '本日': 0.13154185690664655,
 '重點': 0.13154185690664655,
 '老翁': 0.10697681133974268,
 '清潔隊': 0.10665555965403775,
 '診者足': 0.10665555965403775,
 '出海': 0.10608214266665045,
 '婦人': 0.10324506210696097,
 '火勢': 0.10205833725515681,
 '燃燒': 0.10205833725515681,
 '起火': 0.10205833725515681,
 '醫護': 0.10032853492879823,
 '國民黨': 0.09936615090070423,
 '強暴': 0.09532018616423664,
 '防身': 0.09460202037806772,
 '員工': 0.09071852200458383,
 '補習班': 0.08797385334520948,
 '確診者': 0.08704975824704551,
 '標準': 0.08704975824704551,
 '提名': 0.08517098648631792,
 '落海': 0.08486571413332036,
 '楊姓': 0.08486571413332036,
 '群聚': 0.08172718920235438,
 '家庭': 0.08172718920235438,
 '下降': 0.08153420882643382,
 '活動': 0.08108744603834377,
 '報名': 0.08108

In [87]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(smooth_idf=True)
tfidf = vectorizer.fit_transform(paragraphs)
result = pd.DataFrame(tfidf.toarray(), columns=vectorizer.get_feature_names())
keywords = result.sum().sort_values(ascending=False)[:20].index
keywords

Index(['確診', '疫情', '陽性', '警方', '表示', '篩檢', '個案', '相關', '快篩', '衛生局', '醫院', '員工',
       '民眾', '男子', '友人', '救護', '地檢署', '針對', '新增', '女子'],
      dtype='object')

In [90]:
#TF-Density
import math
import numpy as np

def WF_DF_WT(paragraphs):
    wf=dict()
    df=dict()
    wt=0
    for p in paragraphs:
        words = p.split(" ")
        cnt=0
        wt+=1
        for word in words:
            if word not in wf:
                wf[word] = 1
            else:
                wf[word] += 1
            if word not in df:
                df[word] = 1
                cnt+=1
            else:
                if cnt==0:
                    df[word] += 1
    return (wf, df, wt)

def WF_skl(paragraphs):
    cv = CountVectorizer()
    cv_fit = cv.fit_transform(paragraphs)
    word = cv.get_feature_names()
    count = np.asarray(cv_fit.sum(axis=0))[0]
    tf = dict(zip(word, count))
    #print(tf)
    return wf

def DF_skl(paragraph):
    cv = CountVectorizer()
    cv_fit = cv.fit_transform(paragraph)
    word = cv.get_feature_names()
    count = np.asarray(cv_fit.sum(axis=0))[0]
    f = count/np.sum(count)
    tf = dict(zip(word, f))
    #print(tf)
    return tf


def TF(paragraph):
    tf=dict()
    words = paragraph.split(" ")
    total_words = len(words)
    for word in words:
        if word not in tf:
            tf[word] = 1
        else:
            tf[word] += 1
    for k, v in tf.items():
        tf[k] = v / total_words
    return tf

result = dict()
wf, df, wt = WF_DF_WT(paragraphs)
for p in paragraphs:
    tf = TF(p)
    words = p.split(" ")
    for word in words:
            result[word] = (tf[word] * wt * df[word]) / (wf[word] * wf[word])
dict(sorted(result.items(), key=lambda item: item[1], reverse=True))
#dict(sorted(iwf.items(), key=lambda item: item[1], reverse=True))

{'地': 1.8333333333333333,
 '聯合': 1.8333333333333333,
 '新聞網': 1.8333333333333333,
 '讀者': 1.8333333333333333,
 '快速': 1.8333333333333333,
 '整理': 1.8333333333333333,
 '本日': 1.8333333333333333,
 '重點': 1.8333333333333333,
 '巷': 1.03125,
 '墜樓': 1.03125,
 '陳姓': 1.03125,
 '工人': 1.03125,
 '疑似': 1.03125,
 '樓': 1.03125,
 '鷹架': 1.03125,
 '失足': 1.03125,
 '墜落': 1.03125,
 '臟器': 1.03125,
 '外露': 1.03125,
 '明顯': 1.03125,
 '救護人員': 1.03125,
 '採證': 1.03125,
 '勞動': 1.03125,
 '檢查處': 1.03125,
 '派員': 1.03125,
 '贓': 0.8918918918918919,
 '物庫': 0.8918918918918919,
 '健康中心': 0.8918918918918919,
 '在地': 0.8918918918918919,
 '檢署': 0.8918918918918919,
 '完成': 0.8918918918918919,
 '分別': 0.6226415094339622,
 '鄉市': 0.6226415094339622,
 '接到': 0.6226415094339622,
 '發布': 0.6226415094339622,
 '藤山步道': 0.6226415094339622,
 '健走': 0.6226415094339622,
 '鳥早餐': 0.6226415094339622,
 '夫球': 0.6226415094339622,
 '練習場': 0.6226415094339622,
 '藤山': 0.6226415094339622,
 '步道': 0.6226415094339622,
 '住': 0.6226415094339622,
 '噴藥': 0.622641509433